In [3]:
from pydantic import BaseModel, field_serializer

- `when_used`: by default the custom serializer is always used, but we have other options available:
    - `always`: the default, serializer is executed when serializing either to a dict or to JSON,
    - `unless-none`: serializer is not used if the value is None,
    - `json`: serializer is only used when serializing to JSON,
    - `json-unless-none`: serializer used when serializing to JSON, unless the value is None

In [4]:
from datetime import datetime

In [5]:
class Model(BaseModel):
    dt: datetime | None = None
    
    @field_serializer('dt', when_used='always')
    def serialize_name(self, value):
        print(f'type={type(value)}')
        return value

In [6]:
m = Model(dt="2024-08-23T12:00:00")
m

Model(dt=datetime.datetime(2024, 8, 23, 12, 0))

In [7]:
m.model_dump()

type=<class 'datetime.datetime'>


{'dt': datetime.datetime(2024, 8, 23, 12, 0)}

In [8]:
m.model_dump_json()

type=<class 'datetime.datetime'>


'{"dt":"2024-08-23T12:00:00"}'

In [9]:
m = Model(db=None)

In [10]:
m.model_dump()

type=<class 'NoneType'>


{'dt': None}

In [11]:
m.model_dump_json()

type=<class 'NoneType'>


'{"dt":null}'

In [12]:
class Model(BaseModel):
    dt: datetime | None = None
    
    @field_serializer('dt', when_used='unless-none')
    def serialize_name(self, value):
        print(f'type={type(value)}')
        return value

In [14]:
m = Model(dt="2024-08-23T12:00:00")
m.model_dump()

type=<class 'datetime.datetime'>


{'dt': datetime.datetime(2024, 8, 23, 12, 0)}

In [15]:
m.model_dump_json()

type=<class 'datetime.datetime'>


'{"dt":"2024-08-23T12:00:00"}'

In [16]:
m = Model(dt=None)
m.model_dump()

{'dt': None}

In [17]:
m.model_dump_json()

'{"dt":null}'

In [18]:
dt = datetime(2020, 1, 1, 12, 0, 0)

In [19]:
dt

datetime.datetime(2020, 1, 1, 12, 0)

In [20]:
dt.isoformat()

'2020-01-01T12:00:00'

In [21]:
dt.strftime("%Y/%-m/%-d %I:%M %p")

'2020/1/1 12:00 PM'

In [24]:
class Model(BaseModel):
    dt: datetime | None = None
    
    @field_serializer('dt', when_used='json-unless-none')
    def serialize_name(self, value: datetime):
        print(f'type={type(value)}')
        return value.strftime("%Y/%-m/%-d %I:%M %p")

In [25]:
m = Model(dt="2024-08-23T12:00:00")
m.model_dump()

{'dt': datetime.datetime(2024, 8, 23, 12, 0)}

In [26]:
m.model_dump_json()

type=<class 'datetime.datetime'>


'{"dt":"2024/8/23 12:00 PM"}'

In [27]:
from pydantic import FieldSerializationInfo

In [28]:
class Model(BaseModel):
    dt: datetime | None = None
    
    @field_serializer("dt", when_used='unless-none')
    def dt_serializer(self, value, info: FieldSerializationInfo):
        print(f"info={info}")
        return value

In [29]:
m = Model(dt=datetime(2020, 1, 1))

In [30]:
m

Model(dt=datetime.datetime(2020, 1, 1, 0, 0))

In [31]:
m.model_dump()

info=SerializationInfo(include=None, exclude=None, context=None, mode='python', by_alias=False, exclude_unset=False, exclude_defaults=False, exclude_none=False, round_trip=False, serialize_as_any=False)


{'dt': datetime.datetime(2020, 1, 1, 0, 0)}

In [32]:
m.model_dump_json()

info=SerializationInfo(include=None, exclude=None, context=None, mode='json', by_alias=False, exclude_unset=False, exclude_defaults=False, exclude_none=False, round_trip=False, serialize_as_any=False)


'{"dt":"2020-01-01T00:00:00"}'

In [35]:
class Model(BaseModel):
    dt: datetime | None = None
    
    @field_serializer("dt", when_used='unless-none')
    def dt_serializer(self, value, info: FieldSerializationInfo):
        print(f"info={info}")
        print(f"is_json={info.mode_is_json()}")
        return value

In [36]:
m = Model(dt=datetime(2020, 1, 1))
m.model_dump()

info=SerializationInfo(include=None, exclude=None, context=None, mode='python', by_alias=False, exclude_unset=False, exclude_defaults=False, exclude_none=False, round_trip=False, serialize_as_any=False)
is_json=False


{'dt': datetime.datetime(2020, 1, 1, 0, 0)}

In [37]:
from datetime import datetime
import pytz

In [38]:
dt=datetime.now()
dt

datetime.datetime(2024, 8, 23, 13, 9, 14, 871736)

In [40]:
dt=datetime.now(pytz.utc)
dt

datetime.datetime(2024, 8, 23, 11, 9, 28, 538085, tzinfo=<UTC>)

In [41]:
dt.isoformat()

'2024-08-23T11:09:28.538085+00:00'

In [43]:
def make_utc(dt: datetime) -> datetime:
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt

In [44]:
dt = make_utc(datetime.now())

In [45]:
dt

datetime.datetime(2024, 8, 23, 13, 13, 3, 776813, tzinfo=<UTC>)

In [46]:
dt.isoformat()

'2024-08-23T13:13:03.776813+00:00'

In [47]:
make_utc(datetime.now(pytz.utc))

datetime.datetime(2024, 8, 23, 11, 14, 11, 526602, tzinfo=<UTC>)

In [48]:
dt.strftime("%Y-%m-%dT%H:%M:%SZ")

'2024-08-23T13:13:03Z'

In [49]:
def dt_utc_json_serializer(dt: datetime) -> str:
    dt = make_utc(dt)
    return dt.strftime("%Y-%m-%dT%H:%M:%SZ")

In [50]:
class Model(BaseModel):
    dt: datetime | None = None
    
    @field_serializer("dt", when_used='unless-none')
    def dt_serializer(self, dt, info: FieldSerializationInfo):
        if info.mode_is_json():
            return dt_utc_json_serializer(dt)
        return make_utc(dt)

In [51]:
m = Model(dt=datetime(2020, 1, 1))

In [52]:
m

Model(dt=datetime.datetime(2020, 1, 1, 0, 0))

In [53]:
m.model_dump()

{'dt': datetime.datetime(2020, 1, 1, 0, 0, tzinfo=<UTC>)}

In [54]:
m.model_dump_json()

'{"dt":"2020-01-01T00:00:00Z"}'

In [56]:
eastern = pytz.timezone("US/Eastern")
dt = eastern.localize(datetime(2020, 1, 1))

In [57]:
dt

datetime.datetime(2020, 1, 1, 0, 0, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>)

In [58]:
dt.isoformat()

'2020-01-01T00:00:00-05:00'

In [59]:
m = Model(dt=dt)

In [60]:
m

Model(dt=datetime.datetime(2020, 1, 1, 0, 0, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>))

In [61]:
m.model_dump()

{'dt': datetime.datetime(2020, 1, 1, 5, 0, tzinfo=<UTC>)}

In [62]:
m.model_dump_json()

'{"dt":"2020-01-01T05:00:00Z"}'